# Ensemble SWIFT model runs

## About this document


In [ ]:
from swift2.doc_helper import pkg_versions_info

print(pkg_versions_info("This document was generated from a jupyter notebook"))

## Imports

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import swift2.doc_helper as std
import swift2.parameteriser as sp
import swift2.play_record as spr
import uchronia.sample_data as usd
import xarray as xr

from swift2.const import CATCHMENT_FLOWRATE_VARID
from swift2.simulation import get_subarea_ids
from swift2.utils import mk_full_data_id, paste0
from uchronia.data_set import datasets_summaries, get_dataset_ids

In [ ]:
%matplotlib inline

## Synthetic catchment

Let's create a test catchment with a few subareas. Since we will work in a simulation mode, not calibration, we can afford a fairly arbitrary structure. 

A catchment structure can be captured with a set of items about subareas, links and nodes, and the connectivity of links and nodes

In [ ]:
runoff_model='GR4J'

In [ ]:
node_ids=paste0('n', [i+1 for i in range(6)])
link_ids = paste0('lnk', [i+1 for i in range(5)])
node_names = paste0(node_ids, '_name')
link_names = paste0(link_ids, '_name')
from_node = paste0('n', [2,5,4,3,1])
to_node = paste0('n', [6,2,2,4,4])
areas_km2 = np.array([1.2, 2.3, 4.4, 2.2, 1.5])

In [ ]:
simulation = std.create_catchment(node_ids, node_names, link_ids, link_names, from_node, to_node, runoff_model, areas_km2)

In [ ]:
simulation.describe()

## Input data management

Working with ensemble time series is complicated.

The package `uchronia` includes facilities to access time series from a "library", akin to what you would do to manage books. This hides a lot of the lower level code for reading and writing file. To an extent, the python package `xarray` overlaps with the features of these `uchronia` data libraries, but do not fully supersede them.

Let's load a predefined data library with data for the Upper Murray river.

In [ ]:
if not 'SWIFT_TEST_DIR' in os.environ:
    os.environ['SWIFT_TEST_DIR'] = os.path.expanduser('~/data/documentation') 

In [ ]:
doc_data_path = usd.sample_data_dir()
data_path = os.path.join(doc_data_path, 'UpperMurray')

In [ ]:
data_library = usd.sample_time_series_library('upper murray')

In [ ]:
data_library

In [ ]:
data_ids = data_library.get_dataset_ids()
data_ids

In [ ]:
data_library.datasets_summaries()

The sample catchment structure is obviously not the real "Upper Murray". For the sake of a didactic example, let's set the same inputs across all the subareas.

In [ ]:
rain_obs = data_library.get_dataset('rain_obs').to_xarray()

In [ ]:
print(rain_obs)

In [ ]:
rain_obs = rain_obs.where(rain_obs >= 0)

In [ ]:
rain_obs.time[:-10].values

Note that the rainfall is hourly, but the pet is daily. This will matter later for the simulation

In [ ]:
pet_obs = data_library.get_dataset('pet_obs').to_xarray()
pet_obs.time[:-10]

### Setting simulation inputs by reading from a data library

swift simulations have a `play_inputs` (with an s) method designed to retrieve inputs from a library. 

In [ ]:
simulation.play_inputs?

`play_inputs` can accept vectorised arguments, which is handy for cases like the following:

> For each precipitation model input, use the same time series 'rain_obs' from the data library.

In [ ]:
precip_ids = mk_full_data_id( 'subarea', get_subarea_ids(simulation), 'P')
evapIds = mk_full_data_id( 'subarea', get_subarea_ids(simulation), 'E')
precip_ids, evapIds

In [ ]:
def _rep(x): return np.repeat(x, len(precip_ids))
simulation.play_inputs(data_library, precip_ids, _rep('rain_obs'), _rep(''))

We noted that the pet_obs is a daily series, not hourly as per . `swift2` can disaggregate on the fly, using the 'daily_to_hourly' method when assigning inputs to the simulation. This saves a lot of tedium!

In [ ]:
simulation.play_inputs(data_library, evapIds, _rep('pet_obs'), _rep('daily_to_hourly'))
# And the flow rate we will record
outflow_id = CATCHMENT_FLOWRATE_VARID

Given the information from the input data, let's define a suitable simulation time span. We have define an ensemble simulation where we will do a warmup simulation on a single input (no "ensemble") for 3 years or so, then five days of an ensemble simulation.

In [ ]:
from cinterop.timeseries import as_timestamp
 
s = as_timestamp('2007-01-01')
e = as_timestamp('2010-08-01 20')
s_hot = as_timestamp('2010-08-01 21')
e_hot = as_timestamp('2010-08-05 21')

## Warmup the simulation to get 'hot' states

First, before demonstrating ensemble forecasting simulations, let's demonstrate how we can get a snapshot of the model states at a point in time and restore it later on, hot-starting further simulation.

We deliberately get into details here to illustrate how to capture states, and run simulation without or without state reset.

In [ ]:
simulation.set_simulation_span(start=s, end=e_hot)
simulation.record_state(outflow_id)
simulation.exec_simulation()
baseline = simulation.get_recorded(outflow_id)

In [ ]:
baseline = baseline.squeeze(drop=True).sel(time = slice(s_hot, e_hot))

In [ ]:
baseline.plot(figsize=(10,5))
plt.title("streamflow with long term simulation, slice to the end of the series")

In [ ]:
simulation.set_simulation_span(start=s, end=e)
simulation.exec_simulation()
snapshot = simulation.snapshot_state()

We can execute a simulation over the new time span, but requesting model states to NOT be reset. If we compare with a simulation where, as per default, the states are reset before the first time step, we notice a difference:

In [ ]:
simulation.set_simulation_span(start=s_hot, end=e_hot)
simulation.exec_simulation(reset_initial_states = False)
noReset = simulation.get_recorded(outflow_id)
simulation.exec_simulation(reset_initial_states = True)
withReset = simulation.get_recorded(outflow_id)

In [ ]:
noReset = noReset.squeeze(drop=True)

x = xr.concat([noReset,withReset], dim=pd.Index(['no reset','reset'], name='scenario')).squeeze(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(x.time.values, x.sel(scenario='no reset'), linewidth=2, label='No reset')
ax.plot(x.time.values, x.sel(scenario='reset'), linewidth=2, label='Reset')
ax.legend()
plt.show()

The simulation hot-started and run with no reset is like the previous long simulation baseline. If we reset the states to zero, we even have ho streamflow yet produced over these 5 days...

## Ensemble forecasts

Now let'd ready the simulation to do ensemble forecasts. We define a list `inputMap` such that keys are the names of ensemble forecast time series found in `data_library` and the values is one or more of the model properties found in the simulation. In this instance we use the same series for all model precipitation inputs in `precip_ids` 

In [ ]:
simulation.reset_model_states()
simulation.set_states(snapshot)

In [ ]:
inputMap = {'rain_fcast_ens':precip_ids}

In [ ]:
ems = simulation.create_ensemble_forecast_simulation(
    data_library, 
    start=s_hot, 
    end=e_hot, 
    input_map=inputMap, 
    lead_time=(24*2+23), 
    ensemble_size=100, 
    n_time_steps_between_forecasts=24)

In [ ]:
ems

`ems` is an ensemble forecast simulation object, which is an augmentation of the `Simulation` object that deals with non-ensemble simulation. It is very important to note that whenever possible, the object methods are named identically, just that the time series in and out of the simulations are of higher dimension.    

In [ ]:
ems.get_simulation_span()

In [ ]:
ems.record_state(outflow_id)
ems.exec_simulation()
forecasts = ems.get_recorded_ensemble_forecast(outflow_id)

In [ ]:
type(forecasts)

We have four forecast issue times:

In [ ]:
forecasts.time_index()

We can retrieve the first forecast issues at '2010-08-01 21:00:00' by indexing

In [ ]:
flow_forecasts = forecasts[0]

In [ ]:
flow_forecasts

## Visualisation

Let's visualise each of these successive ensemble forecasts. We define a function to determine and visualise the quantiles:

In [ ]:
def plot_ensemble_forecast(flow_forecasts, issue_date):
    q = flow_forecasts.quantile([0.05, .25, .5, .75, 0.95], 'ensemble')
    fig, ax = plt.subplots(figsize=(10,5))
    ax.fill_between(q.time.values, q.sel(quantile=0.05), q.sel(quantile=0.95), alpha=0.3, label='Perc. 50-95')
    ax.fill_between(q.time.values, q.sel(quantile=0.25), q.sel(quantile=.75), alpha=0.5, label='Perc. 25-75')
    ax._get_lines.get_next_color()  # Hack to get different line
    ax.plot(q.time.values, q.sel(quantile=.5), linewidth=2, label='Median')
    ax.legend()
    dd = pd.Timestamp(issue_date).strftime('%Y-%m-%dT%H')
    plt.title(f"Ensemble forecast streamflow {dd}")
    plt.ylabel("Streamflow (m3/s)")
    plt.show()

In [ ]:
issue_dates = forecasts.time_index().values

In [ ]:
plot_ensemble_forecast(forecasts[0], issue_dates[0])

In [ ]:
plot_ensemble_forecast(forecasts[1], issue_dates[1])

In [ ]:
plot_ensemble_forecast(forecasts[2], issue_dates[2])

In [ ]:
plot_ensemble_forecast(forecasts[3], issue_dates[3])

## Appendix

### Data library sample definition

The sample data library used in this vignette is defined by a YAML file defining where time series (or ensemble time series) are on disk in netcdf files. Note that one series can be in several netCDF files, and conversely it is possible to define multiple series (e.g. per station) in one file.

```yaml
pet_fcast_ens:
  Type: single
  Id: pet_fcast_ens
  Storage:
    Type: single_nc_file
    File: ./Fct_Data/Upper_Murray_pet_clim_1990_2010.nc
    Identifier: 1
    Var: pet_der
pet_obs:
  Type: single
  Id: pet_obs
  Storage:
    Type: single_nc_file
    File: ./Obs_data/Upper_Murray_pet_24h_89_2012.nc
    Identifier: 1
    Var: pet_der
rain_obs:
  Type: single
  Id: rain_obs
  Storage:
    Type: single_nc_file
    File: ./Obs_data/Upper_Murray_rain_1hr.nc
    Identifier: 1
    Var: rain_der
rain_fcast_ens:
  Type: ts_ensemble_ts
  Id: rain_fcast_ens
  # The following 6 values are placeholders - may not yet be used by time series library
  TimeStep: 24:00:00
  Start: 2010-08-01T21:00:00
  Length: 5
  EnsembleSize: 1000
  EnsembleLength: 240
  EnsembleTimeStep: 01:00:00
  Storage:
    Type: multiple_nc_files_filename_date_pattern
    File: ./Fct_Data/Upper_Murray_F1_1_{0}21_shuffle.nc
    Var: rain_fcast_ens
    Identifier: 1
    Index: 0
```
